In [1]:
import os

In [2]:
os.chdir(r"C:\Users\HP\Python\LC_ByR")

Corpus extraido de [aquí](https://www.kaggle.com/leangab/poe-short-stories-corpuscsv) 

# Paquetería

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from re import compile
import re
from collections import defaultdict, OrderedDict, Counter
from itertools import chain, combinations
import operator
import string

In [4]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Funciones

## Función de tokenización

In [5]:
regex = compile('[_{}();\.,+:!\[0-9\]$\*"&#\/¡!¿?·''`“”‘’]')                 
    
def tokenize(text):
    """
    Función de tokenización.
    
    Arguments
    ---------
    text : str
        Cadena de texto que se tokenizará
        
    Returns
    -------
    tokens : list
        Lista de tokens
    """

    
    #Retiramos signos
    text= regex.sub(' ',text.lower())
    tok = text.split(' ')
    
    #Pasamos los tokens limpios a la lista tokens
    tokens = []
    for t in tok:
        #Retiramos ' al principio y al final de las string
        #No retiramos todas las ' pues hay palabras que no se entienderian sin '
        if "'" in t:
            a = re.sub("^'",'',t)
            a = re.sub("'$",'',a)
            tokens.append(a)
        #Retiramos las string vacias, pues nuestro corpus las genera por su estructura
        elif t == '':
            pass
        else:
            tokens.append(t)

    return tokens


## Función para frecuencias de tipos
A partir de la lista de tokens

In [6]:
def freq(tokens):
    """
    Función que calcula la frecuencia absoluta de palabras en una lista.
    
    Arguments
    ---------
    tokens : list
        Documento tokenizado
        
    Returns
    -------
    fr : defaultdict
        Diccionario de tokens (tipos) con frecuencias absolutas
    """
    fr = defaultdict(int)
    
    #Conteo de tipos en diccionario, con key igual al tipo dividido por ' ', más </w>
    for i in tokens:
        fr[' '.join(i)+' </w>'] += 1
    
    return fr
    

## Función para frecuencias de caracteres
A partir de diccionario de frecuencias por tipos

In [7]:
def freq_w(dicc):
    """
    Función que calcula frecuencia absoluta de caracteres y total de palabras.
    
    Arguments
    ---------
    dicc : dict
        Diccionario de las frecuencias de las palabras (tipos)
        
    Returns
    -------
    freq_o : OrderedDict
        Diccionario ordenado de las frecuencias de los carateres y cantidad de palabras
    """
    freq = defaultdict(int)
    
    #Contabilizamos palabras y carácteres
    for w, f in dicc.items():
        #Conteo de palabras
        if '</w>' in w:
            freq['</w>'] += f
            w = re.sub(' </w>','',w)
        #Conteo de caracteres
        for ch in w.split():
            freq[ch] += f
    
    #Ordenamos el diccionnario teninendo como primer key a '</w>'
    freq_o = OrderedDict()
    freq_o['</w>']=freq['</w>']
    del freq['</w>']
    freq_o.update(dict(sorted(freq.items(), key = operator.itemgetter(1), reverse=True)))
    
    return freq_o

## Función para obtener el par de simbolos consecutivos más común y unirlos en el diccionario de frecuencias de tipos
A partir de un diccionario de frecuencias de tipos, y un alfabeto con subwords y sus frecuencias

In [8]:
def par_com(dic_freq,al):
    """
    Función para unir par de caracteres más común, y calcular su frecuencia absoluta
    
    Arguments
    ---------
    dic_freq : dict
        Diccionario de frecuencias de palabras
    al : dict
        Lista de carácteres 'básicos' (subwords) con fecuencias
        
    Returns
    -------
    prod : list
        Lista con:
            [0] dic_fr_ord: palabras y frecuencias
            [1] alf_ord: subwords y frecuencias
            [2] tupla con par más común y su frecuencia
            
            
    """
    
    alf = al.copy()
    dic_fr = dic_freq.copy()
    alf.pop('</w>',None)
    
    
    
    voc = defaultdict(int)
    keys = defaultdict(list)
    
    #Contabilizamos la frecuencia de caracteres básicos consecutivos
    for word in dic_fr:
        w = re.sub(' </w>','',word)
        w = w.split()
        for i in range(len(w)-1):
            voc[w[i],w[i+1]] += dic_fr[word]
            keys[(w[i],w[i+1])] += [word]
    
    #Par de caracteres consecutivos más común
    max_key = max(voc.items(), key=operator.itemgetter(1))[0]
    pri, sec = max_key
    
    #Diccionario con el par de caracteres con mayor frecuencia unidos
    dic_fr0 = {word : fr for word, fr in dic_fr.items() if word not in keys[max_key]}
    for word in keys[max_key]:
        w = word.split()
        c=0
        for i in range(len(w)-1):
            if w[i]==pri and w[i+1]==sec:
                w[i] = pri+sec
                del w[i+1]
                c += 1
            if i+1 > len(w)-(2+c):
                break
        dic_fr0[' '.join(w)] = dic_fr[word] 
    
    
    #Ordenando diccionario de palabras por frecuencia
    dic_fr_ord = OrderedDict()
    dic_fr_ord.update(dict(sorted(dic_fr0.items(), key = operator.itemgetter(1), reverse=True)))

    #Actualizando diccionnario de subwords
    alf[pri+sec] = voc[max_key]
    alf[pri] -= voc[max_key]
    alf[sec] -= voc[max_key]
    
    #Ordenando diccionario de subwords por frecuencia
    alf_ord = OrderedDict()
    alf_ord.update(dict(sorted(alf.items(), key = operator.itemgetter(1), reverse=True)))
    
    prod = [dic_fr_ord, alf_ord, (max_key,'Frecuencia: '+str(voc[max_key]))]
    
    #dic_fr_ord: palabras y frecuencias
    #alf_ord: subwords y frecuencias
    #tupla con par más común y su frecuencia
    
    return prod

## Función de sustitución

In [9]:
def sus(corpus, dicc_s):
    """
    Función que calcula frecuencia absoluta de caracteres y total de palabras.
    
    Arguments
    ---------
    corpus : dict
        Corpus
    dicc_s : dict
        Diccionario con número de iteración como clave y tupla con regla de sustitución como valor
        
    Returns
    -------
    n_corpus : dict
        Corpus con la sustitucion dada por dicc_s
    """
    
    c_corpus = corpus.copy()
    
    #Sustituimos las subwords en el corpus
    for i in reversed(range(len(dicc_s))):
        orig = dicc_s[i][0]
        dest = dicc_s[i][1]
        for tit, tx in c_corpus.items():
            c_corpus[tit] = re.sub(orig,dest,tx)
    
    return c_corpus

# Clase _BPE_Tokenization_ 
Para aplicar la tokenización BPE sobre un corpus en forma de diccionario

In [10]:
class BPE_Tokenization(object): 

    def  __init__ (self, corpus):
            
        """
        Clase para tokenización BPE.
        
        Arguments
        ---------
        
        corpus : dict
            Colección de documentos 
            {title:text}
        """
        for i in corpus:
            corpus[i] = corpus[i].lower()
        self.corpus = corpus
        self.frecuencias = None
        self.vocabulario = None
        self.tokens = None
        self.pares = None
        self.asignacion = dict()
        self.iteraciones = 0
        #Curpus con subwords sustituidas
        self.ncorpus = None
        
        
    def tokenizar(self,iteraciones):
        
        self.iteraciones = iteraciones
        
        #Inicializando los atributos para cada invocacion del metodo tokenizar()
        self.frecuencias = None
        self.vocabulario = None
        self.tokens = []
        self.pares = OrderedDict()
        self.asignacion = dict()
        
        #Tokenizando corpus
        for doc in self.corpus:
            self.tokens += tokenize(self.corpus[doc])
        
        #Calculando frecuencias absolutas de tipos y caracteres 
        self.frecuencias = freq(self.tokens)
        self.vocabulario = freq_w(self.frecuencias)
        
        #Pares más comunes por iteración
        for i in range(self.iteraciones):
            self.frecuencias, self.vocabulario, par = par_com(self.frecuencias, self.vocabulario)
            self.pares.update({i:par})
        
            
        #Letras con las que vamos a sustituir las subwords
        abc = ' '.join(string.ascii_uppercase).split()
        
        
        if self.iteraciones>len(abc):
            #Si el número de iteraciones supera el total de letras 
            #que tenemos para sustituir las subwords no hacemos la sustitucioó
            print('Se han generado más de ',len(abc),' iteraciones')
        else:
            #Sustituyendo subwords por letras del abecedario en mayusculas
            for i in range(self.iteraciones):
                self.asignacion[i] = (''.join(self.pares[i][0]),abc[i])
            self.ncorpus = sus(self.corpus, self.asignacion)
            
        
            

# Tokenización BPE

In [11]:
data = pd.read_csv(r"Poe.csv")

Corpus = {data.title[i]:data.text[i] for i in range(data.shape[0])}

In [12]:
prueba = BPE_Tokenization(Corpus)

In [13]:
prueba.tokenizar(7)

In [14]:
#Pares más comunes por iteración
prueba.pares

OrderedDict([(0, (('t', 'h'), 'Frecuencia: 48570')),
             (1, (('th', 'e'), 'Frecuencia: 31317')),
             (2, (('i', 'n'), 'Frecuencia: 28875')),
             (3, (('a', 'n'), 'Frecuencia: 22216')),
             (4, (('e', 'r'), 'Frecuencia: 21554')),
             (5, (('o', 'n'), 'Frecuencia: 18171')),
             (6, (('r', 'e'), 'Frecuencia: 17718'))])

In [15]:
#Regla de sustitución
prueba.asignacion

{0: ('th', 'A'),
 1: ('the', 'B'),
 2: ('in', 'C'),
 3: ('an', 'D'),
 4: ('er', 'E'),
 5: ('on', 'F'),
 6: ('re', 'G')}

In [16]:
#Corpus con sustitución
tit = list(prueba.ncorpus.keys())
print('Titulo:\n','\t'+tit[0]+'\n','Extracto:\n','\t'+prueba.ncorpus[tit[0]],sep='')

Titulo:
	A DESCENT INTO THE MAELSTRÖM
Extracto:
	B ways of god C natuG, as C providence, aG not as our ways; nor aG B models Aat we frame Dy way commensurate to B vastness, profundity, Dd unsearchableness of his works, which have a depA C Bm gGatE AD B well of democritus. joseph glDville.    we had now Gached B summit of B loftiest crag. for some mCutes B old mD seemed too much exhausted to speak.     “not lFg ago,” said he at lengA, “Dd i could have guided you F Ais route as well as B youngest of my sFs; but, about AGe years past, BG happened to me D event such as nevE happened to mortal mD—or at least such as no mD evE survived to tell of—Dd B six hours of deadly tEror which i Bn enduGd have broken me up body Dd soul. you suppose me a vEy old mD—but i am not. it took less AD a sCgle day to chDge Bse hairs from a jetty black to white, to weaken my limbs, Dd to unstrCg my nEves, so Aat i tGmble at B least exEtiF, Dd am frightened at a shadow. do you know i cD scarcely look ovE Ais litt

In [17]:
#Frecuencias de simbolos básicos
prueba.vocabulario

OrderedDict([('e', 122038),
             ('a', 95371),
             ('o', 94812),
             ('t', 93404),
             ('s', 91463),
             ('i', 81302),
             ('d', 62460),
             ('l', 60435),
             ('r', 47352),
             ('u', 44276),
             ('c', 40901),
             ('h', 40262),
             ('m', 39851),
             ('f', 38066),
             ('n', 36159),
             ('the', 31317),
             ('w', 30926),
             ('p', 29961),
             ('in', 28875),
             ('y', 28838),
             ('g', 27703),
             ('b', 23623),
             ('an', 22216),
             ('er', 21554),
             ('on', 18171),
             ('re', 17718),
             ('th', 17253),
             ('v', 16248),
             ('k', 7553),
             ('—', 4926),
             ('x', 3206),
             ('-', 2517),
             ('q', 1775),
             ('j', 1430),
             ('z', 1063),
             ('é', 31),
             ('ö', 19),
     

In [18]:
#Frecuencia de palabras
prueba.frecuencias

OrderedDict([('the </w>', 24634),
             ('o f </w>', 14800),
             ('an d </w>', 9390),
             ('t o </w>', 7769),
             ('a </w>', 7704),
             ('in </w>', 6746),
             ('i </w>', 6004),
             ('i t </w>', 3860),
             ('th a t </w>', 3838),
             ('w a s </w>', 3738),
             ('w i th </w>', 2848),
             ('m y </w>', 2750),
             ('i s </w>', 2692),
             ('a s </w>', 2605),
             ('a t </w>', 2573),
             ('w h i c h </w>', 2439),
             ('h i s </w>', 2256),
             ('h a d </w>', 2176),
             ('h e </w>', 2162),
             ('f o r </w>', 2125),
             ('n o t </w>', 2032),
             ('th i s </w>', 2021),
             ('b y </w>', 1975),
             ('b e </w>', 1815),
             ('b u t </w>', 1772),
             ('h a v e </w>', 1737),
             ('u p on </w>', 1615),
             ('f r o m </w>', 1574),
             ('a l l </w>', 1451),
     